In [1]:
import pandas as pd
import json
import datetime as dt
import os
import sys
import numpy as np
# load_dotenv()
# from dotenv import load_dotenv, dotenv_values

In [2]:
# For recreateability, not everyone is going to have this, so if recreate = True, then it will look for google form data

recreate = True

# File Reading / List population

In [3]:
#Set directory, get list of files in raw data directory, so that we can loop through them
directory = '../data/raw_data'
raw_data_files = os.listdir(directory)
#Create list of dfs, that will hold pointers to each json -> df.
activity_df_list = []
sleep_df_list = []
skin_df_list = []
hrv_df_list = []
brv_df_list = []
#list of dfs from above, for referencing
list_of_dfs = [activity_df_list, sleep_df_list, skin_df_list, hrv_df_list, brv_df_list]
list_of_str = ['get_activities', 'get_sleep', 'get_skin', 'get_hrv', 'get_br']
list_of_norm = ['activities', 'sleep', 'tempSkin', 'hrv', 'br']
#Initialise the excel data dfs
sleep_xsl_df = None
stress_xsl_df = None
mfp_df = None
gf_df = None



In [4]:
#Converting json files in raw_data and then creating a df for each of them, adding them to a list

for index, ldf in enumerate(list_of_dfs):
    
    #looping through files list, and creating a list of jsons by loading them all
    list_of_dfs[index] = [json.load(open(directory + "/" + f)) for f in raw_data_files if f.startswith(list_of_str[index]) and f.endswith('.json')]
    
    #Loop and apply json_normalize on all files in ldf
    list_of_dfs[index][:] = map(lambda x: pd.json_normalize(x[list_of_norm[index]]), list_of_dfs[index])
    
    #Convert normalized json dicts to dfs
    list_of_dfs[index][:] = map(lambda x: pd.DataFrame.from_dict(x), list_of_dfs[index])
    #print(list_of_dfs[index][0].info())

# Activity Data Cleaning

In [5]:
# Loop through activity files normalize, pd them, drop cols, drop more cols (might need a catch), concat all dfs ,
# loop through and transform data, create column names, change types

In [6]:
#First lets remove all the columns that we deemed not necessary.

columns_remove = ['logId','activityLevel','logType','caloriesLink','heartRateLink','tcxLink','lastModified','hasGps','manualValuesSpecified.calories','manualValuesSpecified.distance','manualValuesSpecified.steps','activeZoneMinutes.totalMinutes','activeZoneMinutes.minutesInHeartRateZones','distance','speed','pace','distanceUnit', 'source.type', 'source.id', 'source.url', 'source.trackerFeatures', 'source.name', 'inProgress', 'customHeartRateZones']
#Loop through list of dfs, and call the .drop func and remove listed columns above
list_of_dfs[0][:] = [df.drop(columns=[col for col in columns_remove if col in df.columns], axis=1) for df in list_of_dfs[0]]

In [7]:
#Now concat all the data frames together

list_of_dfs[0] = pd.concat(list_of_dfs[0], ignore_index = True)

In [8]:
#Now expand some of the cols within the df

#Give list_of_dfs[0] an alias for simplicity
activity_df = list_of_dfs[0]

#Initialize new column names in df
base_column_string = 'hrz'
activity_df['hrz_OutofRange_calories'] = None
activity_df['hrz_FatBurn_calories'] = None
activity_df['hrz_Cardio_calories'] = None
activity_df['hrz_Peak_calories'] = None

activity_df['hrz_OutofRange_minutes'] = None
activity_df['hrz_FatBurn_minutes'] = None
activity_df['hrz_Cardio_minutes'] = None
activity_df['hrz_Peak_minutes'] = None

#loop through df
for index in range(0, len(activity_df['heartRateZones'])):
    #get list of heartratezones from nested json structure
    data_list = activity_df['heartRateZones'].iloc[index]
    
    #loop through the kv pair in each item in list mentioned above
    for small_dict in data_list:
        #generate what column the data will be placed in by the value 
        generated_base_col_string = base_column_string + '_' + small_dict['name']
        generated_base_col_string = generated_base_col_string.replace(" ", "")
        
        #get calorie value
        cal_val = small_dict['caloriesOut']
        #insert calories by col name generated and particular index
        activity_df.iloc[index, activity_df.columns.get_loc(generated_base_col_string + '_' + 'calories')] = cal_val
        
        #Repeat process above but with minutes per section
        minute_val = small_dict['minutes']
        activity_df.iloc[index, activity_df.columns.get_loc(generated_base_col_string + '_' + 'minutes')] = minute_val
        

activity_df.drop(columns=['heartRateZones'], axis = 1, inplace = True)

In [9]:
#Rename the columns that I outlined in document
rename_act_cols = {"customHeartRateZones" : "custom_hrz", 
                   "intervalWorkoutData.intervalSummaries" : "iwd_intervalSummaries",
                   "intervalWorkoutData.numRepeats" : "iwd_numRepeats"}

list_of_dfs[0].rename(columns = rename_act_cols, inplace = True)

In [10]:
#change data types 
data_type_change = {'hrz_OutofRange_calories' : 'float64',
                    'hrz_FatBurn_calories' : 'float64',
                    'hrz_Cardio_calories' : 'float64',
                    'hrz_Peak_calories' : 'float64',
                    'hrz_OutofRange_minutes' : 'int64',
                    'hrz_FatBurn_minutes' : 'int64',
                    'hrz_Cardio_minutes' : 'int64',
                    'hrz_Peak_minutes' : 'int64'}

list_of_dfs[0] = list_of_dfs[0].astype(data_type_change)

In [11]:
list_of_dfs[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   activityTypeId           294 non-null    int64  
 1   activityName             294 non-null    object 
 2   calories                 294 non-null    int64  
 3   steps                    293 non-null    float64
 4   averageHeartRate         294 non-null    int64  
 5   duration                 294 non-null    int64  
 6   activeDuration           294 non-null    int64  
 7   startTime                294 non-null    object 
 8   originalStartTime        294 non-null    object 
 9   originalDuration         294 non-null    int64  
 10  elevationGain            294 non-null    float64
 11  hasActiveZoneMinutes     294 non-null    bool   
 12  iwd_intervalSummaries    294 non-null    object 
 13  iwd_numRepeats           294 non-null    int64  
 14  hrz_OutofRange_calories  2

# Sleep Data Cleaning

In [12]:
# Loop through sleep logs, keep only certain columns per each, concat all dfs together

In [13]:
# Remove unnecessary columns 
remove_sleep_cols = ['infoCode', 'logId', 'logType', 'minutesAfterWakeup', 'minutesToFallAsleep']

list_of_dfs[1][:] = [df.drop(columns = [col for col in remove_sleep_cols if col in df.columns], axis = 1) for df in list_of_dfs[1]]

In [14]:
# concat all the dfs 

list_of_dfs[1] = pd.concat(list_of_dfs[1], ignore_index = True)

In [15]:
list_of_dfs[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 30 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   dateOfSleep                               293 non-null    object 
 1   duration                                  293 non-null    int64  
 2   efficiency                                293 non-null    int64  
 3   endTime                                   293 non-null    object 
 4   isMainSleep                               293 non-null    bool   
 5   minutesAsleep                             293 non-null    int64  
 6   minutesAwake                              293 non-null    int64  
 7   startTime                                 293 non-null    object 
 8   timeInBed                                 293 non-null    int64  
 9   type                                      293 non-null    object 
 10  levels.data                           

In [16]:
# Read sleep csv, (go through date conversion stuff and only keep necessary things), 
# (dont drop first row, idk why that is there), create datetimekey col and convert to datetime..., perform join, b

In [17]:
#Read csv
sleep_score_df = pd.read_csv("../data/raw_data/takeout/Fitbit/Sleep Score/sleep_score.csv")
sleep_score_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sleep_log_entry_id     276 non-null    int64  
 1   timestamp              276 non-null    object 
 2   overall_score          276 non-null    int64  
 3   composition_score      276 non-null    int64  
 4   revitalization_score   276 non-null    int64  
 5   duration_score         276 non-null    int64  
 6   deep_sleep_in_minutes  276 non-null    int64  
 7   resting_heart_rate     276 non-null    int64  
 8   restlessness           276 non-null    float64
dtypes: float64(1), int64(7), object(1)
memory usage: 19.5+ KB


In [18]:
#Drop columns that are not needed
scd = ['sleep_log_entry_id', 'deep_sleep_in_minutes']

sleep_score_df.drop(columns=scd, axis = 1, inplace = True)

In [19]:
#Create columns in both dfs, to match standard of datetime

#API Format: 2023-11-24T10:51:30.000
#CSV Format: 2023-11-24T10:51:30Z

#Read as string, and get first 19 characters in both, set values for datetime_key column
list_of_dfs[1]['datetime_key'] = list_of_dfs[1]['endTime'].str.slice(start=0, stop=19)
sleep_score_df['datetime_key'] = sleep_score_df['timestamp'].str.slice(start=0, stop=19)

#merge with left join, since API data should always be upto date, and csv download is scheduled for every two months
list_of_dfs[1] = pd.merge(list_of_dfs[1], sleep_score_df, on="datetime_key", how="left")

In [20]:
#Drop unnecessary datetime columns from old ones, since new key has good level of detail
list_of_dfs[1].drop(columns=['timestamp', 'endTime'], axis=1, inplace=True)

In [21]:
list_of_dfs[1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293 entries, 0 to 292
Data columns (total 36 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   dateOfSleep                               293 non-null    object 
 1   duration                                  293 non-null    int64  
 2   efficiency                                293 non-null    int64  
 3   isMainSleep                               293 non-null    bool   
 4   minutesAsleep                             293 non-null    int64  
 5   minutesAwake                              293 non-null    int64  
 6   startTime                                 293 non-null    object 
 7   timeInBed                                 293 non-null    int64  
 8   type                                      293 non-null    object 
 9   levels.data                               293 non-null    object 
 10  levels.shortData                      

# Skin / HRV / BRV Data Cleaning

In [22]:
# For these three, all that needs to be done is concat all the dfs, then keep certain columns, thats all.

In [23]:
#Concat the three in each of their sections

for i in range (2,5):
    list_of_dfs[i] = pd.concat(list_of_dfs[i], ignore_index = True)

In [24]:
#Only need to drop for brv

list_of_dfs[2].drop(columns = ['logType'], axis=1, inplace = True)

In [25]:
print(list_of_dfs[2].info())
print(list_of_dfs[3].info())
print(list_of_dfs[4].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dateTime               277 non-null    object 
 1   value.nightlyRelative  277 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   dateTime          282 non-null    object 
 1   value.dailyRmssd  282 non-null    float64
 2   value.deepRmssd   282 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dateTime             281 n

# Stress Data Cleaning

In [26]:
#Drop unnecessary columns, convert all columns to lower, convert any types if necessary

In [27]:
stress_df = pd.read_csv('../data/raw_data/Takeout/Fitbit/Stress Score/Stress Score.csv')

In [28]:
stress_drop_columns = ['DATE', 'MAX_SLEEP_POINTS', 'MAX_RESPONSIVENESS_POINTS', 'MAX_EXERTION_POINTS', 'STATUS', 'CALCULATION_FAILED']

stress_df.drop(columns = stress_drop_columns, axis = 1, inplace = True)

In [29]:
stress_df.columns = stress_df.columns.str.lower()

In [30]:
stress_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   updated_at             281 non-null    object
 1   stress_score           283 non-null    int64 
 2   sleep_points           283 non-null    int64 
 3   responsiveness_points  283 non-null    int64 
 4   exertion_points        283 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 11.2+ KB


# MyFitnessPal Cleaning

In [31]:
#Change column names as specified in document

In [32]:
#Get filename from list of files in directory
filename = None
for f in raw_data_files:
    if f.startswith("Nutrition"):
        filename = f

mfp_df = pd.read_csv('../data/raw_data/' + filename)

In [33]:
#rename columns 

mfp_rename = {'Date' : 'date', 'Meal' : 'meal', 'Calories' : 'calories', 'Fat (g)' : 'fat_g', 
              'Saturated Fat' : 'sat_fat', 'Polyunsaturated Fat' : 'poly_fat', 'Monounsaturated Fat' : 'mono_fat',
              'Trans Fat' : 'trans_fat', 'Cholesterol' : 'cholesterol', 'Sodium (mg)' : 'sodium_mg', 
              'Potassium' : 'potassium', 'Carbohydrates (g)' : 'carbohydrates_g', 'Fiber' : 'fiber_g', 
              'Sugar' : 'sugar', 'Protein (g)' : 'protein_g', 'Vitamin A' : 'vitamin_a', 'Vitamin C' : 'vitamin_c', 
              'Calcium' : 'calcium', 'Iron' : 'iron'}

mfp_df.rename(columns = mfp_rename, inplace = True)

In [34]:
mfp_df.drop(columns = ['Note'], axis = 1, inplace = True)

In [35]:
mfp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             85 non-null     object 
 1   meal             85 non-null     object 
 2   calories         85 non-null     float64
 3   fat_g            85 non-null     float64
 4   sat_fat          85 non-null     float64
 5   poly_fat         85 non-null     float64
 6   mono_fat         85 non-null     float64
 7   trans_fat        85 non-null     float64
 8   cholesterol      85 non-null     float64
 9   sodium_mg        85 non-null     float64
 10  potassium        85 non-null     float64
 11  carbohydrates_g  85 non-null     float64
 12  fiber_g          85 non-null     float64
 13  sugar            85 non-null     float64
 14  protein_g        85 non-null     float64
 15  vitamin_a        85 non-null     float64
 16  vitamin_c        85 non-null     float64
 17  calcium          8

# Google Form Data Cleaning

In [36]:
# rename columns, expand colum for one of the data cols

In [37]:
if not recreate:
    raise KeyboardInterrupt

In [38]:
gf_df = pd.read_csv('../data/raw_data/Fishbit_GF_Responses.csv')

In [39]:
#Rename the columns
gf_df.columns = ['timestamp','date', 'last_sleep_feel', 'last_sleep_score', 'daily_particular_qualities', 'day_feel', 
                 'ef_status', 'ef_description', 'dissatisfied_status', 'stress_mgmt_score']

In [40]:
print(gf_df.columns)

Index(['timestamp', 'date', 'last_sleep_feel', 'last_sleep_score',
       'daily_particular_qualities', 'day_feel', 'ef_status', 'ef_description',
       'dissatisfied_status', 'stress_mgmt_score'],
      dtype='object')


In [41]:
# split the 'stress_mgmt_score' column, after dropping null
split_cols = gf_df['stress_mgmt_score'].dropna().str.split(',', expand=True)

# assign first value back to stress_mgmt_score
gf_df.loc[split_cols.index, 'stress_mgmt_score'] = split_cols[0]

# assign the rest of the columns to their values 
gf_df.loc[split_cols.index, 'stress_responsiveness'] = split_cols[1]
gf_df.loc[split_cols.index, 'stress_exertion'] = split_cols[2]
gf_df.loc[split_cols.index, 'stress_sleep'] = split_cols[3]

# fill the inbetween values with nan
gf_df[['stress_responsiveness', 'stress_exertion', 'stress_sleep']] = gf_df[['stress_responsiveness', 'stress_exertion', 'stress_sleep']].fillna(np.nan)


In [42]:
datatype_change = {'stress_mgmt_score':'int64',
                   'stress_responsiveness':'int64',
                   'stress_exertion':'int64',
                   'stress_sleep':'int64',
                  }

# gf_df = gf_df.astype(datatype_change)

#Force the types to be Int64, not int64... today I learned that pandas Int64 is different and supports null unlike the int64 based off numpy

gf_df['stress_mgmt_score'] = pd.to_numeric(gf_df['stress_mgmt_score'], errors='coerce').astype('Int64')
gf_df['stress_responsiveness'] = pd.to_numeric(gf_df['stress_responsiveness'], errors='coerce').astype('Int64')
gf_df['stress_exertion'] = pd.to_numeric(gf_df['stress_exertion'], errors='coerce').astype('Int64')
gf_df['stress_sleep'] = pd.to_numeric(gf_df['stress_sleep'], errors='coerce').astype('Int64')

In [43]:
gf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   timestamp                   51 non-null     object 
 1   date                        51 non-null     object 
 2   last_sleep_feel             44 non-null     object 
 3   last_sleep_score            37 non-null     float64
 4   daily_particular_qualities  44 non-null     object 
 5   day_feel                    44 non-null     object 
 6   ef_status                   44 non-null     object 
 7   ef_description              14 non-null     object 
 8   dissatisfied_status         43 non-null     object 
 9   stress_mgmt_score           1 non-null      Int64  
 10  stress_responsiveness       1 non-null      Int64  
 11  stress_exertion             1 non-null      Int64  
 12  stress_sleep                1 non-null      Int64  
dtypes: Int64(4), float64(1), object(8)
me

# Export all DFs to CSV

In [44]:
#Now we just export all of the final dfs as a csv to the interim data folder. 

In [45]:
#Some of the dfs need proper date sorting:

#br dateTime column needs to be sorted
#hrv dateTime col too
#tempSkin dateTime too

#list_of_norm = ['activities', 'sleep', 'tempSkin', 'hrv', 'br']
list_of_dfs[2].sort_values(by="dateTime", inplace=True)
list_of_dfs[3].sort_values(by="dateTime", inplace=True)
list_of_dfs[4].sort_values(by="dateTime", inplace=True)


In [46]:
#Loop through list_of_dfs and convert each index into a csv
for index, df in enumerate(list_of_dfs):
    df.to_csv("../data/interim/initial_clean_" + list_of_norm[index] + ".csv", index=False)

#convert mfp and gf to csv
mfp_df.to_csv("../data/interim/initial_clean_mfp.csv", index = False)
gf_df.to_csv("../data/interim/initial_clean_gf.csv", index = False)
    